In [1]:
import warnings

warnings.filterwarnings('ignore')

import json
from tqdm import tqdm
import pandas as pd
import requests
import re

## split pdb apart

In [2]:
pep_set = pd.read_csv('venom_pep.csv')

In [ ]:
entries = pep_set[pep_set['DBSOURCE'] != 'pdb:']

In [ ]:
entries.shape

## Get pdb code

In [3]:
codes_list = []

In [4]:
def get_data(url):
    

    header = {
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    'From': 'jailann22silvasousa@gmail.com'
    }
    try:
        response = requests.get(url, header)
        response.raise_for_status
        data = response.content
        return data
    except requests.exceptions.HTTPError as err:
        raise SystemError(err)
    

In [ ]:
entries_ids = ['B3EWX2']
# 'P43220', 'P03397','P26349', 'P07839', 'Q40378', 'P03397', 'P03404', 'P01317'

In [5]:
for en in tqdm(pep_set.Entry):

    
    url = f'https://rest.uniprot.org/uniprotkb/search?format=json&fields=structure_3d&query={en}'
    
    strings = str(get_data(url), 'utf-8')
    
    pdb = re.findall(r'\b[1-9]{1}[0-9A-Z]{3}\b', strings)
    resolution = re.findall(r'[0-9]\.[0-9 A]+|"-"', strings)
    
    if not pdb:
        pdb = ['null']
    if not resolution:
        resolution = ['null']
        
    
    for code, res in zip(pdb, resolution):
    
        codes = {
                'entry':en,
                'pdb_code':code,
                'resolution':res
                }
    
        codes_list.append(codes)
    
    
    

print('Finished')

100%|█████████████████████████████████████████| 779/779 [18:32<00:00,  1.43s/it]

Finished


In [ ]:
print(codes_list)

In [6]:
df = pd.DataFrame(codes_list)

In [7]:
df.to_csv('venom_pep_pdbcode.csv', index=False)